<!-- Autoheader begin -->
<hr/>
<div id="navtitle_3_4_py" style="text-align:center; font-size:16px">III.4 Entangling Quantum Gates for Coupled Transmon Qubits</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_3_chiral.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="py_exercise_3_3_chiral.ipynb" style="font-size:13px">III.3 Using Krotov's method to separate chiral molecules</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_1_TLS.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="py_exercise_3_1_TLS.ipynb" style="font-size:13px">III.1 Population Inversion in a Two-Level-System using Krotov's Method</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Julia/jl_exercise_3_4_gate.ipynb">👉 Julia version</a></div>

---
<!-- Autoheader end -->

# Entangling Quantum Gates for Coupled Transmon Qubits

$
\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{braket}[1]{\langle#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2}\mid{#2}\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}
$

In this notebook, we will use the `krotov` package for an optimization towards a perfectly entangling two-qubit gate in a simplified model of two transmon qubits with a shared transmission line. We first consider the direction optimization for a $\Op{O} = \sqrt{\text{iSWAP}}$ gate with a standard square-modulus functional. Then, we perform anoptimization towards a general perfect entangler using the functional demonstrated in [Watts *et al.*, Phys. Rev. A 91, 062306 (2015)](https://michaelgoerz.net/#WattsPRA2015) and [Goerz *et al.*, Phys. Rev. A 91, 062307 (2015)](https://michaelgoerz.net/#GoerzPRA2015)

This notebook builds upon the simpler applications of Krotov's method in [Exercise III.2](py_exercise_3_2_lambda.ipynb) and [Exercise III.3](py_exercise_3_3_chiral.ipynb). In those examples, the optimization had to consider only the time evolution of a single quantum state. In contrast, for the optimization of a quantum gate, the optimization functional needs to account for the evolution of multiple states simultaneously (specifically, the logical two-qubit basis states $\ket{00}$, $\ket{01}$, $\ket{10}$, $\ket{11}$). You will learn in this exercise how to work with this more involved functionals, and how the functional and the dynamics affect the optimization in Krotov's method.

This notebook serves as a nice illustration of using optimal control in a quantum information context, extending the simpler optimizations discussed previously for a two-level-system (i.e. a qubit) in [Exercise II.1](py_exercise_2_1_TLS.ipynb) and [Exercise III.1](py_exercise_3_1_TLS.ipynb).


##  Setup

First, we have to load the `krotov` package an other basic numerical tools, including the `qutip` packaged used to describe quantum objects.

In [ ]:
import krotov
import qutip
import numpy as np
import scipy

For visualization, we will use the `matplotlib` package

In [ ]:
import matplotlib
import matplotlib.pylab as plt

In [ ]:
# Some utilities for showing hints and solutions
from utils.exercise_3_gate import *

## Model

We consider a generic two-qubit Hamiltonian (motivated from the example of two
superconducting transmon qubits, truncated to the logical subspace),

$$
\begin{equation}
  \op{H}(t)
    = - \frac{\omega_1}{2} \op{\sigma}_{z}^{(1)}
      - \frac{\omega_2}{2} \op{\sigma}_{z}^{(2)}
      + 2 J \left(
            \op{\sigma}_{x}^{(1)} \op{\sigma}_{x}^{(2)}
            + \op{\sigma}_{y}^{(1)} \op{\sigma}_{y}^{(2)}
        \right)
      + u(t) \left(
            \op{\sigma}_{x}^{(1)} + \lambda \op{\sigma}_{x}^{(2)}
        \right),
\end{equation}
$$

where $\omega_1$ and $\omega_2$ are the energy level splitting of the
respective qubit, $J$ is the effective coupling strength and $u(t)$ is the
control field. $\lambda$ defines the strength of the qubit-control coupling for
qubit 2, relative to qubit 1.

We use the following parameters:

In [ ]:
w1 = 1.1  # qubit 1 level splitting
w2 = 2.1  # qubit 2 level splitting
J = 0.2  # effective qubit coupling
u0 = 0.3  # initial driving strength
la = 1.1  # relative pulse coupling strength of second qubit
T = 25.0  # final time
nt = 250  # number of time steps

tlist = np.linspace(0, T, nt)

These values are for illustrative purposes only, and do not correspond to any
particular implementation of the superconducting architecture.

In [ ]:
def hamiltonian(Ω, w1=w1, w2=w2, J=J, la=la, u0=u0):
    """Two qubit Hamiltonian

    Args:
        w1 (float): energy separation of the first qubit levels
        w2 (float): energy separation of the second qubit levels
        J (float): effective coupling between both qubits
        la (float): factor that pulse coupling strength differs for second qubit
        u0 (float): constant amplitude of the driving field
    """
    # local qubit Hamiltonians
    Hq1 = 0.5 * w1 * np.diag([-1, 1])
    Hq2 = 0.5 * w2 * np.diag([-1, 1])

    # lift Hamiltonians to joint system operators
    H0 = np.kron(Hq1, np.identity(2)) + np.kron(np.identity(2), Hq2)

    # define the interaction Hamiltonian
    sig_x = np.array([[0, 1], [1, 0]])
    sig_y = np.array([[0, -1j], [1j, 0]])
    Hint = 2 * J * (np.kron(sig_x, sig_x) + np.kron(sig_y, sig_y))
    H0 = H0 + Hint

    # define the drive Hamiltonian
    H1 = np.kron(np.array([[0, 1], [1, 0]]), np.identity(2)) + la * np.kron(
        np.identity(2), np.array([[0, 1], [1, 0]])
    )

    # convert Hamiltonians to QuTiP objects
    H0 = qutip.Qobj(H0)
    H1 = qutip.Qobj(H1)

    return [H0, [H1, Ω]]

The initial guess is defined as

In [ ]:
def eps0(t, args):
    return u0 * krotov.shapes.flattop(
        t, t_start=0, t_stop=T, t_rise=(T / 20), t_fall=(T / 20), func='sinsq'
    )

In [ ]:
def plot_pulse(pulse, tlist):
    fig, ax = plt.subplots()
    if callable(pulse):
        pulse = np.array([pulse(t, args=None) for t in tlist])
    ax.plot(tlist, pulse)
    ax.set_xlabel('time')
    ax.set_ylabel('pulse amplitude')
    plt.show(fig)

In [ ]:
plot_pulse(eps0, tlist)

Finally, we connect the Hamiltonian with this guess pulse.

In [ ]:
H = hamiltonian(eps0, w1=w1, w2=w2, J=J, la=la, u0=u0)

## Logical basis for two-qubit gates

For simplicity, we define the qubits in the *bare* basis, i.e.
ignoring the static coupling $J$.

In [ ]:
psi_00 = qutip.Qobj(np.kron(np.array([1, 0]), np.array([1, 0])))
psi_01 = qutip.Qobj(np.kron(np.array([1, 0]), np.array([0, 1])))
psi_10 = qutip.Qobj(np.kron(np.array([0, 1]), np.array([1, 0])))
psi_11 = qutip.Qobj(np.kron(np.array([0, 1]), np.array([0, 1])))

basis = [psi_00, psi_01, psi_10, psi_11]

## Optimizing for a specific quantum gate

Our target gate is $\Op{O} = \sqrt{\text{iSWAP}}$:

In [ ]:
SQRTISWAP = qutip.qip.operations.sqrtiswap()
SQRTISWAP

The `krotov` package provides a function `get_objectives` which initializes the "objectives" containing the basis states and the target gate.

In [ ]:
objectives = krotov.gate_objectives(
    basis_states=[psi_00, psi_01, psi_10, psi_11], gate=SQRTISWAP, H=H
)
objectives

This is one objective per basis function, with the basis state as the `initial_state` and a `target` state that is the gate applied that basis state. We can verify this for the second objective:

In [ ]:
objectives[1].initial_state

In [ ]:
objectives[1].target

Because *any* state can be expanded in the basis, finding a control field so that the `initial_state` evolves to the `target` for all of the four objectives guarantees that we have successfully implemented the quantum gate.

We can analyze how all of the basis states evolve under the guess controls:

In [ ]:
guess_states = [objectives[i].mesolve(tlist).states[-1] for i in range(4)]

The gate implemented by the guess controls can be found as follows

In [ ]:
U_guess = qutip.Qobj(
    [[basis[i].overlap(guess_states[j]) for i in range(4)] for j in range(4)],
    dims = [[2, 2], [2, 2]]
)

We will optimize these trajectories with a square-modulus functional

$$
J_{T,sm}
= 1 - \Bigg\vert\frac{1}{4}\sum_{k=1}^{4} \underbrace{\langle \Psi_k(T) | \Psi_k^{\text{tgt}}\rangle}_{\equiv \tau_k}\Bigg\vert^2
= 1 - \frac{1}{16} \sum_{k,l=1}^{4} \underbrace{\langle \Psi_l^{\text{tgt}} | \Psi_l(T) \rangle}_{\equiv\tau_l^*} \; \underbrace{\langle \Psi_k(T) | \Psi_k^{\text{tgt}}\rangle}_{\equiv \tau_k}
$$

where $\ket{\Psi_k(T)}$ is the result of forward-propagation the basis state $\ket{\phi_k}$ (where $\ket{\phi_1} = \ket{00}$, $\ket{\phi_2} = \ket{01}$, etc.)

In [ ]:
from krotov.functionals import J_T_sm

The initial value of the functional is

In [ ]:
J_T_sm(guess_states, objectives)

which is the gate error

In [ ]:
1 - abs((U_guess.dag() * SQRTISWAP).tr() / 4)**2

An illustration on the way the pulse update is computed in Krotov's method can be found in part (b) of the figure below.

<img src="../figures/schemes.svg" alt="Schemes" style="width: 1200px;"/>

The figure is taken from from the paper [Goerz *et al.*, Quantum 6, 871 (2022)](https://quantum-journal.org/papers/q-2022-12-07-871/), where more details can be found.

An essential feature of the schemes for Krotov's method is that it involves the
backward propagation of a set of states $\ket{\chi_k}$ with the boundary condition

$$
\ket{\chi_k(T)} = - \frac{\partial J_T}{\partial \bra{\Psi_k(T)}}
$$

The same backward propagation with the same boundary condition can also be used in GRAPE, although we will not explore the use of GRAPE further in this example.

### Problem 1: boundary condition for the backward propagation

In the `krotov` package, a function that provides the $\ket{\chi_k(T)}$ states must be passed to the `krotov.optimize_pulses` function as `chi_constructor`. This is how the functional enters the equations for the iterative optimization in Krotov's algorithm!

For the $J_{T,sm}$ as defined above, you will find that that $\ket{\chi_k(T)}$ are proportional to the target states $\ket{\Psi_k^{\text{tgt}}}$. Calculate with pen and paper the derivative $-\partial J_T / \partial \bra{\Psi_k(T)}$ and fill in the proportionality factor $\alpha$ below.

In [ ]:
def chi_constructor(fw_states_T, objectives, **kwargs):
    τ = np.array([fw_states_T[k].overlap(objectives[k].target) for k in range(4)])
    α = # fill in the proportionality factor
    chi_states = []
    for k in range(4):
        chi_states.append(
            α * objectives[k].target
        )
    return chi_states

In [ ]:
# problem_1.hint

In [ ]:
# problem_1.solution

Now, we collect the full optimization problem containing the list of trajectories, the optimization functional and the definition of `chi_constructor`. We also need some Krotov-specific `pulse_options`, which include the step width `lambda_a` and an update shape that scales the pulse update at each point in time and can be used to ensure the boundary condition that the field has to smoothly switch on from zero at the beginning and smoothly switch off to zero again at the end:

In [ ]:
def S(t):
    """Shape function for the field update"""
    return krotov.shapes.flattop(
        t, t_start=0, t_stop=T, t_rise=T / 20, t_fall=T / 20, func='sinsq'
    )

In [ ]:
pulse_options = {H[1][1]: dict(lambda_a=1.0, update_shape=S)}

In [ ]:
opt_result = krotov.optimize_pulses(
    objectives,
    pulse_options=pulse_options,
    tlist=tlist,
    propagator=krotov.propagators.expm,
    chi_constructor=chi_constructor,
    info_hook=krotov.info_hooks.print_table(
        J_T=krotov.functionals.J_T_sm,
        show_g_a_int_per_pulse=False,
        unicode=False,
    ),
    check_convergence=krotov.convergence.Or(
        krotov.convergence.value_below(1e-2, name="J_T"),
        krotov.convergence.check_monotonic_error,
    ),
    iter_stop=100,
)
opt_result

Take a look at what happens if the `chi_constructor` function is implemented incorrectly. For example, use `τ` instead of `τ.conjugate()`, or try to use the wrong sign.

We can plot the resulting control field:

In [ ]:
plot_pulse(opt_result.optimized_controls[0], tlist)

Would you consider this a good solution? Investigate how changing `lambda_a` influences the features of the optimized control field. You may want to limit the number of iterations in order to gain some quick intuition without having to spend a lot of time waiting for `optimize_pulses` obtain full convergence.

## Maximization of the gate concurrence

Building a quantum computer requires a "universal gate" set. Traditionally, this set consists of a specific two-qubit gate (often CNOT), and all single-qubit gates (under the assumption that single-qubit gates are "easy" to realize). However the universal set does not need to contain CNOT (or some other gate) *specifically*. What matters for universal quantum computing is the ability to create entanglement.

To any two-qubit gate (any 4 × 4 unitary) a so-called "gate concurrence" can be computed, which is the maximum entanglement (i.e. concurrence) of a state that can be obtained by applying the gate to some separable input state. A `concurrence` function is implemented in the `weylchamber` package – the Weyl chamber is a mathematical structure which describes and classifies two-qubit gates in terms of entangling power and equivalence with respect to single-qubit operations. You can find an illustration of the Weyl chamber below.

<img src="../figures/weylchamber.svg" alt="Weyl chamber" style="width: 800px;"/>

In [ ]:
import weylchamber

The gate concurrence is defined in terms of the "Weyl Chamber coordinates" $c_1$, $c_2$, $c_3$ that are the axes in the diagram.

Most of the "standard" two-qubit gates like $\sqrt{\text{iSWAP}}$ and CNOT (points Q and L, respectively, in the diagram) are "perfect entanglers":

In [ ]:
weylchamber.concurrence(*weylchamber.c1c2c3(SQRTISWAP))

(Note: Please ignore any `RuntimeWarning` you might see the first time you run this function.)

In [ ]:
weylchamber.concurrence(*weylchamber.c1c2c3(qutip.qip.operations.cnot()))

The gate concurrence of the identity or any other random $SU(2) \times SU(2)$ matrix (corresponding to single-qubit gates) is zero:

In [ ]:
weylchamber.concurrence(*weylchamber.c1c2c3(np.eye(4)))

In [ ]:
def random_unitary(N):
    H = np.random.rand(N, N)
    return scipy.linalg.expm(1j * (H + H.conjugate().transpose()))

U = qutip.tensor(qutip.Qobj(random_unitary(2)), qutip.Qobj(random_unitary(2)))
weylchamber.concurrence(*weylchamber.c1c2c3(U))

In general, the gate concurrence of a random $4 \times 4$ unitary is a number between 0 and 1; heavily skewing towards 1. Interestingly, the majority of $4 \times 4$ unitaries are perfect entanglers!

In [ ]:
for _ in range(10):
    print(weylchamber.concurrence(*weylchamber.c1c2c3(random_unitary(4))))

This fact makes the gate concurrence an attractive optimization target: by optimizing the entangling power of the two-qubit gate without targeting a *specific* gate, we may identify the perfect entangler that is "easiest" to achieve with the given Hamiltonian.

We can also check that the guess pulse indeed does not yet implement a perfect entangler. So let's get to work!

In [ ]:
weylchamber.concurrence(*weylchamber.c1c2c3(U_guess))

### Problem 2: Boundary condition (χ-states) for concurrence optimization

If we want to optimize the gate concurrence with Krotov's method, we would have to work out the boundary condition for the backward propagation, $\ket{\chi_k(T)} = -\partial J_T / \partial \bra{\Psi_k(T)}$.

One issue is that that `c1c2c3` function takes a $4 \times 4$ matrix `U` as an arguments, whereas the `chi_constructor` must be defined in terms of `fw_states_T`. Luckily, this issue is rather straightforward to solve. The conversion is simply given by

$$
\ket{\Psi_k(T)} = \Op{U} \ket{\phi_k}
\quad \Leftrightarrow \quad
U_{ij} = \braket{\phi_i|\Op{U}|\phi_j} = \braket{\phi_i | \Psi_j(T)}\,,
$$

where the $\ket{\phi_k}$ are the logical basis states and $\ket{\Psi_j(T)}$ are the states in `fw_states_T`.

Beyond that, we would have to look at how the gate concurrence is calculated, either by looking at the original literature, [Kraus, Cirac. Phys. Rev. A 63, 062309 (2001)](https://arxiv.org/abs/quant-ph/0011050) and [Childs *et al.* Phys. Rev. A 68, 052311 (2003)](https://arxiv.org/abs/quant-ph/0307190). Alternatively, you may directly look at the code for the functions `c1c2c3` and `concurrence`:

In [ ]:
??weylchamber.c1c2c3

In [ ]:
??weylchamber.concurrence

Explain with these code snippets why it would be difficult to calculate the derivative $\frac{\partial J_T}{\partial \bra{\Psi_k(T)}}$.

In [ ]:
# problem_2.hint

In [ ]:
# problem_2.solution

Since we cannot optimize the gate concurrence directly, we will have to find an alternative approach. The use of an alternative functional was demonstrated in
[Watts *et. al*, Phys. Rev. A 91, 062306 (2015)](https://michaelgoerz.net/#WattsPRA2015) and [Goerz *et al.*, Phys. Rev. A 91, 062307 (2015)](https://michaelgoerz.net/#GoerzPRA2015)

The basic idea is that the mathematical structure of the two-qubit gates in the Weyl chamber has a *geometric* interpretation. The set of perfect entanglers form a compact polyhedron inside the Weyl chamber (the shaded region in the diagram), and we can optimize for a perfect entangler by minimizing the geometric distance to the surface of that polyhedron.

It can be shown that this geometric distance can be expressed in terms of the "local invariants" $g_3$, $g_2$, $g_3$, which are related to the Weyl chamber coordinates $c_1$, $c_2$, $c_3$, but unlike the Weyl chamber coordinates, the local invariants can be calculated analytically from the gate $\op{U}$.

This allows us to define the following "perfect-entangler functional",

$$
\begin{equation}
  F_{PE} = g_3 \sqrt{g_1^2 + g_2^2} - g_1,
\end{equation}
$$

A list of four objectives that encode the minimization of $F_{PE}$ are
generated by calling the `gate_objectives` function with the canonical basis,
and `"PE"` as target "gate".

In [ ]:
objectives = krotov.gate_objectives(
    basis_states=[psi_00, psi_01, psi_10, psi_11], gate="PE", H=H
)

In [ ]:
objectives

The initial states in these objectives are not the canonical basis states, but a Bell
basis,

In [ ]:
for obj in objectives:
    display(obj.initial_state)

Since we don't know *which* perfect entangler the optimization result will
implement, we cannot associate any "target state" with each objective, and the
`target` attribute is set to the string 'PE'.

We can treat the above objectives as a "black box" - the only important
consideration is that the `chi_constructor` that we will pass to
`optimize_pulses` to calculating the boundary condition for the backwards
propagation,

$$
\begin{equation}
  \ket{\chi_{k}} = \frac{\partial F_{PE}}{\partial \bra{\phi_k}} \Bigg|_{\ket{\phi_{k}(T)}}\,,
\end{equation}
$$

must be consistent with how the `objectives` are set up. For the perfect-entangler functional, the calculation of the $\ket{\chi_{k}}$ is relatively
complicated. The `weylchamber` package contains a suitable routine that
works on the `objectives` exactly as defined above (specifically, under the
assumption that the $\ket{\phi_k}$ are the appropriate Bell states):

In [ ]:
help(weylchamber.perfect_entanglers.make_PE_krotov_chi_constructor)

In [ ]:
chi_constructor = weylchamber.perfect_entanglers.make_PE_krotov_chi_constructor(
    [psi_00, psi_01, psi_10, psi_11]
)

We define the `pulse_options` with the step width `lambda_a` and the `update_shape`, ensuring that the control field is well-behaved at the edges of the time grid:

In [ ]:
pulse_options = {H[1][1]: dict(lambda_a=1.0e2, update_shape=S)}

We will use a custom `info_hook` to analyze each iteration in a bit more detail:

In [ ]:
def print_fidelity(**args):
    basis = [objectives[i].initial_state for i in [0, 1, 2, 3]]
    states = [args['fw_states_T'][i] for i in [0, 1, 2, 3]]
    U = weylchamber.gates.gate(basis, states)
    c1, c2, c3 = weylchamber.coordinates.c1c2c3(weylchamber.from_magic(U))
    g1, g2, g3 = weylchamber.local_invariants.g1g2g3_from_c1c2c3(c1, c2, c3)
    conc = weylchamber.perfect_entanglers.concurrence(c1, c2, c3)
    F_PE = weylchamber.perfect_entanglers.F_PE(g1, g2, g3)
    print("    F_PE: %f\n    gate conc.: %f" % (F_PE, conc))
    return F_PE, [c1, c2, c3]

The values return by this function will be stored in `Result.info_vals`, and we may also use them in a custom convergence check:

In [ ]:
def check_PE(result):
    # extract F_PE from (F_PE, [c1, c2, c3])
    F_PE = result.info_vals[-1][0]
    if F_PE <= 0:
        return "achieved perfect entangler"
    else:
        return None

In [ ]:
opt_result = krotov.optimize_pulses(
    objectives,
    pulse_options=pulse_options,
    tlist=tlist,
    propagator=krotov.propagators.expm,
    chi_constructor=chi_constructor,
    info_hook=krotov.info_hooks.chain(
        krotov.info_hooks.print_debug_information, print_fidelity
    ),
    check_convergence=check_PE,
    iter_stop=20,
)
opt_result

The final optimized control field looks like this:

In [ ]:
plot_pulse(opt_result.optimized_controls[0], tlist)

Note how much easier this optimization was (from a numerical perspective) than the direct optimization towards the $\sqrt{\text{iSWAP}}$ gate!

## Next steps

The [Julia version of this exercise](../Julia/jl_exercise_3_4_gate.ipynb) goes
considerably farther than the simplified model used here: It considers a
logical subsystem embedded in a larger Hilbert space, along with more realistic
parameters. It also uses semi-automatic differentiation to directly optimize
the non-analytic gate concurrence. Furthermore, it contains optimization with GRAPE
in addition to Krotov's method.

You may also continue with [further examples from the documentation of the
`krotov` package](https://qucontrol.github.io/krotov/v1.2.1/09_examples.html)
which are beyond the scope of this tutorial. These include many interesting setups, e.g., the
optimization of quantum gates in open quantum systems.

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_3_4_py)
<!-- Autofooter end -->